# Porównanie Różnych Silników Text-to-Speech (TTS)

Ten notebook przedstawia różne opcje syntezy mowy (TTS) dostępne dla projektu FreeFlow, w tym:
- Google Cloud Text-to-Speech
- Microsoft Azure Cognitive Services
- Amazon Polly
- OpenAI TTS
- Rozwiązania open source

Każdy z tych systemów ma swoje zalety i ograniczenia, które przeanalizujemy pod kątem:
- Jakości głosu
- Wsparcia dla języka polskiego
- Kosztów
- Łatwości integracji
- Możliwości dostosowania

## 1. Wymagane Biblioteki

Zainstalujmy najpierw potrzebne biblioteki:

In [ ]:
# Instalacja wymaganych bibliotek
!pip install google-cloud-texttospeech azure-cognitiveservices-speech boto3 openai TTS

In [ ]:
# Import bibliotek
import os
import google.cloud.texttospeech as gtts
import azure.cognitiveservices.speech as azure_speech
import boto3
import openai
from TTS.api import TTS

# Przykładowy tekst do testów
test_text = "Witaj w FreeFlow! Jak mogę ci dzisiaj pomóc?"

## 2. Google Cloud Text-to-Speech

Google Cloud oferuje wysokiej jakości syntezę mowy z obsługą wielu języków, w tym polskiego. Szczególnie dobrze radzi sobie z intonacją i naturalnością mowy.

In [ ]:
# Konfiguracja klienta Google Cloud TTS
def google_tts(text, output_file="output_google.mp3"):
    client = gtts.TextToSpeechClient()
    
    # Konfiguracja głosu i parametrów audio
    synthesis_input = gtts.SynthesisInput(text=text)
    voice = gtts.VoiceSelectionParams(
        language_code="pl-PL",
        name="pl-PL-Wavenet-B",  # Możesz wybrać inny głos
        ssml_gender=gtts.SsmlVoiceGender.NEUTRAL
    )
    audio_config = gtts.AudioConfig(
        audio_encoding=gtts.AudioEncoding.MP3
    )
    
    # Generowanie audio
    response = client.synthesize_speech(
        input=synthesis_input,
        voice=voice,
        audio_config=audio_config
    )
    
    # Zapisanie do pliku
    with open(output_file, "wb") as out:
        out.write(response.audio_content)
    
    return output_file

# Test Google TTS
try:
    output_file = google_tts(test_text)
    print(f"Utworzono plik: {output_file}")
except Exception as e:
    print(f"Błąd podczas generowania Google TTS: {e}")

## 3. OpenAI TTS

OpenAI oferuje nowoczesny system TTS z wysoką jakością głosu i naturalnym brzmieniem. Aktualnie używany w projekcie.

In [ ]:
# Konfiguracja OpenAI TTS
def openai_tts(text, output_file="output_openai.mp3", voice="nova"):
    try:
        response = openai.audio.speech.create(
            model="tts-1",
            voice=voice,  # dostępne: alloy, echo, fable, onyx, nova, shimmer
            input=text
        )
        response.stream_to_file(output_file)
        return output_file
    except Exception as e:
        print(f"Błąd podczas generowania OpenAI TTS: {e}")
        return None

# Test różnych głosów OpenAI
voices = ["alloy", "echo", "fable", "onyx", "nova", "shimmer"]
for voice in voices:
    try:
        output_file = openai_tts(test_text, f"output_openai_{voice}.mp3", voice)
        print(f"Utworzono plik dla głosu {voice}: {output_file}")
    except Exception as e:
        print(f"Błąd dla głosu {voice}: {e}")

## 4. Coqui TTS (Open Source)

Coqui TTS to open-source'owe rozwiązanie, które pozwala na lokalną generację mowy bez konieczności korzystania z usług chmurowych.

In [ ]:
# Konfiguracja Coqui TTS
def coqui_tts(text, output_file="output_coqui.mp3"):
    try:
        # Inicjalizacja modelu
        tts = TTS(model_name="tts_models/multilingual/multi-dataset/xtts_v2")
        
        # Generowanie audio
        tts.tts_to_file(text=text, 
                        file_path=output_file,
                        language="pl")
        return output_file
    except Exception as e:
        print(f"Błąd podczas generowania Coqui TTS: {e}")
        return None

# Test Coqui TTS
try:
    output_file = coqui_tts(test_text)
    print(f"Utworzono plik: {output_file}")
except Exception as e:
    print(f"Błąd: {e}")

## 5. Porównanie Usług TTS

Poniżej przedstawiamy porównanie różnych usług TTS pod względem kluczowych parametrów:

| Usługa | Jakość Głosu | Wsparcie PL | Koszt | Łatwość Integracji | Offline |
|--------|--------------|-------------|-------|-------------------|----------|
| OpenAI TTS | Bardzo dobra | Częściowe | $0.015/1K znaki | Łatwa | Nie |
| Google TTS | Bardzo dobra | Pełne | $4/1M znaki | Średnia | Nie |
| Coqui TTS | Dobra | Pełne | Darmowy | Trudniejsza | Tak |

### Rekomendacja
1. **OpenAI TTS** (obecnie używane) - najlepsza opcja dla małej i średniej skali, dobra jakość i proste API
2. **Google TTS** - dobre wsparcie dla polskiego, ale droższe przy większej skali
3. **Coqui TTS** - dobre jako backup lub dla offline deployments